<div style="width: 100%; overflow: hidden;">
    <div style="display: block;margin-left: auto;margin-right: auto; width: 100%;"><img src="./1.png"  /></div>
    </a>
</div>

<h2 style="font-family:Avantgarde; font-size:2em; text-align: center;">Projeto 1 - Fusão de Informação e Análise de Dados</h2>


<h4 style="font-family:Avantgarde; font-size:1.75em; text-align: center;">Mestrado em Engenharia e Ciência de Dados</h4>
<h4 style="font-family:Avantgarde; font-size:1.3em; text-align: center;">Ano Letivo 2023/2024</h4>

<p style = "font-family:Avantgarde; font-size:1.1em; text-align: center;"> <b>Mariana Lopes Paulino - 2020190448</b></p>
<p style = "font-family:Avantgarde; font-size:1.1em; text-align: center;"> <b>Rui Alexandre Coelho Tapadinhas - 2018283200</b></p>

<p style="font-family:Avantgarde; font-size:1.25em;"> <b> Tarefas </b> </p>
<p style="font-family:Avantgarde; font-size:1em;"> 1. Criar uma função que implementa o modelo dinâmico do robot; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 2. Criar uma função que implementa o modelo de observação; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 3. Criar uma função que representa a aquisição de dados dos sensores; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 4. Criar uma função que implementa o Extended Kalman Filter (EKF); </p>
<p style="font-family:Avantgarde; font-size:1em;"> 5. Criar uma função que implementa o Unscented Kalman Filter (UKF); </p>
<p style="font-family:Avantgarde; font-size:1em;"> 6. Especificar os vários parâmetros dos filtros e de simulação do robot; </p>
<p style="font-family:Avantgarde; font-size:1em;"> 7. Simular o movimento do robot e representar dinamicamente a sua localização, considerando as trajetórias e a elipse indicadas na Figura 1 </p>


<p style="font-family:Avantgarde; font-size:1.15em;"> <b> Informações Relevantes </b> </p>
<p style="font-family:Avantgarde; font-size:1em;"> O robot tem 4 estados no instante k. xk, yk, Φk, vt.  (x, y) é a posição 2D do robot. Φ é a sua orientação e v a sua velocidade. </p>

<p style="font-family:Avantgarde; font-size:1.5em;"> 0. Import Libraries </p>

In [98]:
from math import sqrt, cos, sin, ceil, atan2, pi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
from sklearn.metrics import mean_squared_error as mse, r2_score as r2

<p style="font-family:Avantgarde; font-size:1.5em;"> 1. Modelo Dinâmico </p>

<p style="font-family:Avantgarde; font-size:1em;"> O modelo não linear dinâmico do robot é definido pela expressão: </p>

<p style="font-family:Avantgarde; font-size:1em; text-align: center;"> <b> x(k+1) = A*x(k) + B*u(k) + w(k) </b> </p>

In [99]:
def input():
    v = 1.0  # [m/s]
    yawrate = 0.1  # [rad/s]
    u = np.array([[v, yawrate]]).T
    return u

In [100]:
def observation(xTrue, xd, u):

    xTrue = dynamic_motion_model(xTrue, u)

    # add noise to gps x-y
    z = observation_model(xTrue) + GPS_Noise @ np.random.randn(2, 1)

    # add noise to input
    ud = u + Input_Noise @ np.random.randn(2, 1)

    xd = dynamic_motion_model(xd, ud)

    return xTrue, z, xd, ud

In [101]:
#TODO: Add noise to the motion model
def dynamic_motion_model(x, u):
    phi = x[2, 0]
    #Formula x(k+1) = A*x(k) + B*u(k) + w(k)

    #Creates a 4x4 matrix where the elements belonging to the principal diagonal are 1,1,1,0.
    A = np.diag([1.0, 1.0, 1.0, 0])
    #print(A)

    B = np.array([[cos(phi) * delta_t, 0],
                  [sin(phi) * delta_t, 0],
                  [0.0, delta_t],
                  [1.0, 0.0]])
    #print(B)

    # Noise
    

    x = A @ x + B @ u

    return x

<p style="font-family:Avantgarde; font-size:1.5em;"> 2. Modelo de Observação </p>

In [102]:
#TODO: Add noise to the observation model
def observation_model(x):
    #Formula z(k) = C*x(k) + r(k)

    C = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0]])
    #print(C)

    z = C @ x

    return z

<p style="font-family:Avantgarde; font-size:1.5em;"> 3. Aquisição de Dados dos Sensores </p>

<p style="font-family:Avantgarde; font-size:1.5em;"> 4. Extended Kalman Filter (EKF) </p>

In [103]:
def jacobA(x, u):
    v = u[0, 0]
    phi = x[2, 0]
    
    jacob_matrix = np.array([[1, 0, -v * np.sin(phi) * delta_t, np.cos(phi) * delta_t],
                   [0, 1, v * np.cos(phi) * delta_t, np.sin(phi) * delta_t],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]], dtype=float)
    
    return jacob_matrix

<p style="font-family:Avantgarde; font-size:1.5em;"> 5. Unscented Kalman Filter (UKF)</p>

In [104]:
def generate_sigma_points(xEst, PEst, gamma):
    sigma = xEst
    Psqrt = sqrtm(PEst)
    n = len(xEst[:, 0])
    # Positive direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst + gamma * Psqrt[:, i:i + 1]))

    # Negative direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst - gamma * Psqrt[:, i:i + 1]))

    return sigma

In [105]:
def predict_sigma_motion(sigma, u):
    """
        Sigma Points prediction with motion model
    """
    for i in range(sigma.shape[1]):
        sigma[:, i:i + 1] = dynamic_motion_model(sigma[:, i:i + 1], u)

    return sigma

In [106]:
def predict_sigma_observation(sigma):
    """
        Sigma Points prediction with observation model
    """
    for i in range(sigma.shape[1]):
        sigma[0:2, i] = observation_model(sigma[:, i])

    sigma = sigma[0:2, :]

    return sigma

In [107]:
def calc_sigma_covariance(x, sigma, wc, Pi):
    nSigma = sigma.shape[1]
    d = sigma - x[0:sigma.shape[0]]
    P = Pi
    for i in range(nSigma):
        P = P + wc[0, i] * d[:, i:i + 1] @ d[:, i:i + 1].T
    print(P)
    return P

In [108]:
def calc_pxz(sigma, x, z_sigma, zb, wc):
    nSigma = sigma.shape[1]
    dx = sigma - x
    dz = z_sigma - zb[0:2]
    P = np.zeros((dx.shape[0], dz.shape[0]))

    for i in range(nSigma):
        P = P + wc[0, i] * dx[:, i:i + 1] @ dz[:, i:i + 1].T
    
    return P

In [109]:
def ukf_estimation(xEst, PEst, z, u, wm, wc, gamma):
    #  Predict
    sigma = generate_sigma_points(xEst, PEst, gamma)
    sigma = predict_sigma_motion(sigma, u)
    xPred = (wm @ sigma.T).T
    PPred = calc_sigma_covariance(xPred, sigma, wc, Q)

    #  Update
    zPred = observation_model(xPred)
    y = z - zPred
    sigma = generate_sigma_points(xPred, PPred, gamma)
    zb = (wm @ sigma.T).T
    #print(zb.shape)
    z_sigma = predict_sigma_observation(sigma)
    st = calc_sigma_covariance(zb, z_sigma, wc, R)
    Pxz = calc_pxz(sigma, xPred, z_sigma, zb, wc)
    K = Pxz @ np.linalg.inv(st)
    xEst = xPred + K @ y
    PEst = PPred - K @ st @ K.T

    return xEst, PEst

In [110]:
def plot_covariance_ellipse(xEst, PEst):
    Pxy = PEst[0:2, 0:2]
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * pi + 0.1, 0.1)
    a = sqrt(eigval[bigind])
    b = sqrt(eigval[smallind])
    x = [a * cos(it) for it in t]
    y = [b * sin(it) for it in t]
    angle = atan2(eigvec[bigind, 1], eigvec[bigind, 0])
    R = np.array([[cos(angle), sin(angle)],
                  [-sin(angle), cos(angle)]])
    fx = R.dot(np.array([[x, y]]))
    px = np.array(fx[0, :] + xEst[0, 0]).flatten()
    py = np.array(fx[1, :] + xEst[1, 0]).flatten()
    plt.plot(px, py, "--r")

<p style="font-family:Avantgarde; font-size:1.5em;"> 7. Simulação do Movimento do Robot </p>

In [111]:
%matplotlib notebook

def main():
    print("Robot simulation start!")

    nx = 4  # State Vector [x y yaw v]'

    xEst = np.zeros((nx, 1))
    xTrue = np.zeros((nx, 1))
    PEst = np.eye(nx)

    wm, wc, gamma = setup_ukf(nx)

    xDR = np.zeros((nx, 1))  # Dead reckoning

    # history
    hxEst = xEst
    hxTrue = xTrue
    hxDR = xTrue
    hz = np.zeros((2, 1))
    
    time = 0.0

    fig = plt.figure()
    fig.show()
    seconds_simulation=50.0
    while seconds_simulation >= time:
        time += delta_t
        u = input()

        xTrue, z, xDR, ud = observation(xTrue, xDR, u)

        xEst, PEst = ukf_estimation(xEst, PEst, z, ud, wm, wc, gamma)
        #print(xEst)
        #print(PEst)

        # store data history
        hxEst = np.hstack((hxEst, xEst))
        hxDR = np.hstack((hxDR, xDR))
        hxTrue = np.hstack((hxTrue, xTrue))
        hz = np.vstack((hz, z))
             
        if show_animation:
            plt.cla()
            # for stopping simulation with the esc key.
            plt.gcf().canvas.mpl_connect('key_release_event',
                    lambda event: [exit(0) if event.key == 'escape' else None])

            plt.plot(hz[0, :], hz[1, :], ".g")
            plt.plot(hxTrue[0, :].flatten(),
                     hxTrue[1, :].flatten(), "-b")
            plt.plot(hxDR[0, :].flatten(),
                     hxDR[1, :].flatten(), "-k")
            plt.plot(hxEst[0, :].flatten(),
                     hxEst[1, :].flatten(), "-r")
            plot_covariance_ellipse(xEst, PEst)
            plt.axis("equal")
            plt.grid(True)
            fig.canvas.draw()
            plt.pause(0.001)

In [112]:
main()

Robot simulation start!


<IPython.core.display.Javascript object>

[[ 1.01890298e+00 -6.57597378e-15  1.27792637e-13  1.06791047e-11]
 [-6.13188457e-15  1.02780591e+00  1.33438799e-01 -8.20845551e-25]
 [ 1.27792637e-13  1.33438799e-01  1.00030462e+00  1.38679518e-22]
 [ 1.06791047e-11  2.28108175e-24  1.37025157e-22  1.00000000e+00]]
[[ 2.01890298e+00 -8.05233111e-15]
 [-8.05233111e-15  2.02780591e+00]]
[[ 5.17990168e-01 -6.75296390e-04 -6.28262935e-03  9.86283582e-12]
 [-6.75296390e-04  5.34201861e-01  1.46660633e-01  7.66355706e-13]
 [-6.28262935e-03  1.46660633e-01  9.91828358e-01  1.67008410e-21]
 [ 9.86283582e-12  7.66355706e-13  1.66677538e-21  1.00000000e+00]]
[[ 1.51799017e+00 -6.75296390e-04]
 [-6.75296390e-04  1.53420186e+00]]
[[ 3.51337350e-01 -4.49859828e-04 -5.07225978e-03 -1.89326064e-13]
 [-4.49859828e-04  3.61070849e-01  1.09312476e-01 -1.34679621e-14]
 [-5.07225978e-03  1.09312476e-01  9.78087615e-01 -2.78097925e-22]
 [-1.89326064e-13 -1.34679621e-14 -2.78925105e-22  1.00000000e+00]]
[[ 1.35133735e+00 -4.49859828e-04]
 [-4.49859828e-0